<a href="https://colab.research.google.com/github/yvrjsharma/MLApps_with_Gradio/blob/main/Langchain_Gradio_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --q langchain==0.0.55 requests openai transformers faiss-cpu

In [ ]:
! pip install --q gradio

In [ ]:
from langchain.llms import OpenAI
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.docstore.document import Document
import requests
import gradio as gr 

import pathlib
import subprocess
import tempfile

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "xx"
#os.environ["SERPAPI_API_KEY"] = "xx"

In [ ]:
# using a vector space for our search
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.faiss import FAISS

In [ ]:
#using this
chain = load_qa_with_sources_chain(OpenAI(temperature=0))
#chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="map_reduce")

def print_answer(question):
    print(
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=4),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]
    )

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
source_chunks = []
splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)

In [ ]:
#not atm
for source in sources:
    for chunk in splitter.split_text(source.page_content):
        source_chunks.append(Document(page_content=chunk, metadata=source.metadata))

search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings())

In [ ]:
# https://github.com/gradio-app/gradio
repo_owner, repo_name = 'gradio-app', 'gradio'

In [ ]:
#new github md func
def get_github_docs(repo_owner, repo_name):
    with tempfile.TemporaryDirectory() as d:
        subprocess.check_call(
            f"git clone https://github.com/{repo_owner}/{repo_name}.git .",
            cwd=d,
            shell=True,
        )
        git_sha = (
            subprocess.check_output("git rev-parse HEAD", shell=True, cwd=d)
            .decode("utf-8")
            .strip()
        )
        repo_path = pathlib.Path(d)
        markdown_files = list(repo_path.rglob("*.md")) + list(
            repo_path.rglob("*.mdx")
        )
        for markdown_file in markdown_files:
            try:
                with open(markdown_file, "r") as f:
                    relative_path = markdown_file.relative_to(repo_path)
                    github_url = f"https://github.com/{repo_owner}/{repo_name}/blob/{git_sha}/{relative_path}"
                    yield Document(page_content=f.read(), metadata={"source": github_url})
            except FileNotFoundError:
                print(f"Could not open file: {markdown_file}")


In [ ]:
sources = get_github_docs("gradio-app", "gradio")

In [ ]:
sources

<generator object get_github_docs at 0x7f9fa5700350>

In [ ]:
#not now
for source in sources:
  print("*****")
  print(source.page_content)

In [ ]:
#source_chunks = []
#splitter = CharacterTextSplitter(separator=" ", chunk_size=1024, chunk_overlap=0)
for source in sources:
    for chunk in splitter.split_text(source.page_content):
        source_chunks.append(Document(page_content=chunk, metadata=source.metadata))

In [ ]:
search_index = FAISS.from_documents(source_chunks, OpenAIEmbeddings())

In [ ]:
#new
source_chunks

[Document(page_content='<!-- DO NOT EDIT THIS FILE DIRECTLY. INSTEAD EDIT THE `readme_template.md` OR `guides/1)getting_started/1)quickstart.md` TEMPLATES AND THEN RUN `render_readme.py` SCRIPT. -->\n\n<div align="center">\n\n  [<img src="readme_files/gradio.svg" alt="gradio" width=300>](https://gradio.app)<br>\n  <em>Build & share delightful machine learning apps easily</em>\n\n  [![gradio-backend](https://github.com/gradio-app/gradio/actions/workflows/backend.yml/badge.svg)](https://github.com/gradio-app/gradio/actions/workflows/backend.yml)\n  [![gradio-ui](https://github.com/gradio-app/gradio/actions/workflows/ui.yml/badge.svg)](https://github.com/gradio-app/gradio/actions/workflows/ui.yml)  \n  [<img src="https://codecov.io/gh/gradio-app/gradio/branch/master/graph/badge.svg" alt="codecov">](https://app.codecov.io/gh/gradio-app/gradio)\n  [![PyPI](https://img.shields.io/pypi/v/gradio)](https://pypi.org/project/gradio/)\n  [![PyPI downloads](https://img.shields.io/pypi/dm/gradio)](h

In [ ]:
#old
source_chunks

In [ ]:
#new
print_answer("how do you create a scatter plot in gradio?") 

 You can create a scatter plot in gradio by using the `gr.ScatterPlot` component, which accepts a pandas dataframe and some optional configuration parameters.
SOURCES: https://github.com/gradio-app/gradio/blob/431a987d612d5e43097898d13523fb72bdf84214/CHANGELOG.md


In [ ]:
#new 
print_answer("how can i build a demo for handwritten digit recognition?") 

 I don't know.
SOURCES: N/A


In [ ]:
#new
print_answer("how to build a demo for sketching?") 

 This guide covers how to build a pictionary app (step-by-step): 

1. [Set up the Sketch Recognition Model](#1-set-up-the-sketch-recognition-model)
2. [Define a `predict` function](#2-define-a-predict-function)
3. [Create a Gradio Interface](#3-create-a-gradio-interface)

SOURCES: https://github.com/gradio-app/gradio/blob/431a987d612d5e43097898d13523fb72bdf84214/guides/06_other-tutorials/building-a-pictionary-app.md


In [ ]:
import pickle
#saving FAISS search index to disk
with open("search_index.pickle", "wb") as f:
        pickle.dump(search_index, f) #(FAISS.from_documents(source_chunks, OpenAIEmbeddings()), f)

In [ ]:
#loading FAISS search index from disk
with open("search_index.pickle", "rb") as f:
    search_index = pickle.load(f)

In [ ]:
gr.HTML("""<div style="text-align: center; max-width: 700px; margin: 0 auto;">
        <div
        style="
            display: inline-flex;
            align-items: center;
            gap: 0.8rem;
            font-size: 1.75rem;
        "
        >
        <h1 style="font-weight: 900; margin-bottom: 7px; margin-top: 5px;">
            Gradio QandA - LangChain Bot
        </h1>
        </div>
        <p style="margin-bottom: 10px; font-size: 94%">
        Hi, I'm a Q and A Gradio expert bot, start by typing in your OpenAI API key, questions/issues you are facing in your Gradio implementations and then press enter.<br>
        <a href="https://huggingface.co/spaces/ysharma/InstructPix2Pix_Chatbot?duplicate=true"><img src="https://bit.ly/3gLdBN6" alt="Duplicate Space"></a>Duplicate Space with GPU Upgrade for fast Inference & no queue<br> 
        Built using <a href="https://langchain.readthedocs.io/en/latest/" target="_blank">LangChain</a> and <a href="https://github.com/gradio-app/gradio" target="_blank">Gradio Github repo</a>
        </p>
    </div>""")

## Gradio demo

In [ ]:
response = print_answer("How to provide examples in Gradio Blocks layout?")
response

 Examples in Gradio Blocks layout can be provided using the `gr.Examples` component, which takes two required parameters: `examples` which takes in a nested list, and `inputs` which takes in a component or list of components.
SOURCES: https://github.com/gradio-app/gradio/blob/792289cdf77ed0dfaa6592a58b106bd66e743e38/guides/01_getting-started/02_key-features.md


' Examples in Gradio Blocks layout can be provided using the `gr.Examples` component, which takes two required parameters: `examples` which takes in a nested list, and `inputs` which takes in a component or list of components.\nSOURCES: https://github.com/gradio-app/gradio/blob/792289cdf77ed0dfaa6592a58b106bd66e743e38/guides/01_getting-started/02_key-features.md'

In [ ]:
def print_answer(question):
    response = (
        chain(
            {
                "input_documents": search_index.similarity_search(question, k=4),
                "question": question,
            },
            return_only_outputs=True,
        )["output_text"]
    )
    #print(response)
    if len(r.split('\n')[-1].split())>2:
        response = response.split('\n')[0] + ', '.join([' <a href="' + response.split('\n')[-1].split()[i] + '" target="_blank"><u>Click Link' + str(i) + '</u></a>' for i in range(1,len(response.split('\n')[-1].split()))])
    else: 
        response = response.split('\n')[0] + ' <a href="' + response.split('\n')[-1].split() + '" target="_blank"><u>Click Link</u></a>'
    return response

In [ ]:
!pip install --q gradio
import gradio as gr

In [ ]:

def chat(message, history):
    history = history or []
    message = message.lower()
    response = print_answer(message) #"I don't know"
    history.append((message, response))
    return history, history

chatbot = gr.Chatbot().style(color_map=("green", "pink"))
#demo = gr.Interface(chat, ["text", "state"], [chatbot, "state"], allow_flagging="never",)
with gr.Blocks() as demo:
  gr.Markdown("""'Yes, it is possible to auto log inputs and outputs using the "allow_flagging" parameter. <a href="https://github.com/gradio-app/gradio/blob/792289cdf77ed0dfaa6592a58b106bd66e743e38/guides/06_other-tutorials/using-flagging.md," target="_blank"><u>Click Link1</u></a>,  <a href="https://github.com/gradio-app/gradio/blob/792289cdf77ed0dfaa6592a58b106bd66e743e38/guides/01_getting-started/02_key-features.md" target="_blank"><u>Click Link2</u></a>'""")
  with gr.Row():
    question = gr.Textbox(label = 'Type in your questions about Gradio here', placeholder = 'What is the role of "every" argument in a component')
    openai_api_key = gr.Textbox()
  state = gr.State()
  chatbot = gr.Chatbot()
  question.submit(chat, [question, state], [chatbot, state])
  #button_gradio_bot = gr.Button('Get Help!')
  #button_gradio_bot.click(chat, [question, state], [chatbot, state])

if __name__ == "__main__":
    demo.launch()


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>